In [1]:
# Steps 
# 1. Train sklearn SVM with both RBF and linear kernel to benchmark 
# 1a. Use a different dataset if this is too easy (breast cancer or housing dataset)
# 2. Make sure implementation of SMO isn't terrible (check)
# 2a. Make sure IP implementation isn't terrible compared to sklearn benchmark (half check, it still sucks)
# 3. Implement IP yourself (done)
# 3b. Implement SMO yourself
# 4. Write derivations (done)
# 5. Run experiments

In [1]:
import sklearn
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn import svm, datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

/Users/aneesh/anaconda3/envs/opt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/aneesh/anaconda3/envs/opt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/aneesh/anaconda3/envs/opt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/aneesh/anaconda3/envs/opt/lib/python3.6/site-

In [54]:
# Dataset Setup

# Returns: 
# X_train, y_train, X_test, y_test
def get_data(dataset):
    np.random.seed(27)
    if dataset == 'iris':
        iris = datasets.load_iris()
        X = iris.data[:, :2]
        y = iris.target
    elif dataset == 'digits':
        digits = datasets.load_digits()
        X = digits.data[:]
        y = digits.target
    elif dataset == 'wine':
        wine = datasets.load_wine()
        X = wine.data[:]
        y = wine.target
    elif dataset == 'cancer':
        cancer = datasets.load_breast_cancer()
        X = cancer.data[:]
        y = cancer.target
    elif dataset == 'mnist':
        SUBSAMPLE_NUM = 500
        X, y = datasets.fetch_openml('mnist_784', version=1, return_X_y=True)
        sub_indices = (y == '1') | (y == '3')
        X = X[sub_indices]
        y = y[sub_indices]
        X = X[:SUBSAMPLE_NUM, :]
        y = y[:SUBSAMPLE_NUM]
    elif dataset == 'nonlinear':
        X = np.random.randn(200, 2)
        y = np.logical_xor(X[:, 0] > 0,
                               X[:, 1] > 0)
        y = np.where(y, 1, -1)
    else:
        raise ValueError("Dataset string not recognized")
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    return X_train, X_test, y_train, y_test

whole_X_train, X_test, whole_y_train, y_test = get_data(dataset='mnist')
# split further into train and val 
X_train, X_val, y_train, y_val = train_test_split(whole_X_train, whole_y_train, test_size=0.15, random_state=42)

In [55]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(382, 784)
(68, 784)
(50, 784)


### Label preprocessing

In [57]:
y_train_floats = [float(i) for i in y_train]
y_val_floats = [float(i) for i in y_val]
y_test_floats = [float(i) for i in y_test]
whole_y_train_floats = [float(i) for i in whole_y_train]

label_map = {}
unique_labels = set(y_train_floats)
assert(len(unique_labels) == 2)
flag = True       
for label in unique_labels:
    if flag:
        label_map[label] = 1
        flag = False
    else:
        label_map[label] = -1
print(label_map)

y_train_floats = np.array([label_map[label] for label in y_train_floats])
y_val_floats = np.array([label_map[label] for label in y_val_floats])
y_test_floats = np.array([label_map[label] for label in y_test_floats])
whole_y_train_floats = np.array([label_map[label] for label in whole_y_train_floats])

{1.0: 1, 3.0: -1}


#### Load kernels

In [6]:
from kernels import kernel_rbf, kernel_poly
from log_barrier.svm import Barrier 
from log_barrier.svm import SVMClassifier
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### SMO Implementation

In [47]:
from smo.smo import SMO

In [49]:
best_acc = float('-inf')
best_g = None
best_C = None 

best_accs = []
best_gs = []
best_Cs = []

print("True labels: ", y_test_floats)
for C in np.logspace(-1, 3, num=5):
    for g in np.logspace(-10, -6, num = 20):
        print("Testing for C = {}, g = {}".format(C, g))
        kernel = kernel_rbf(g)
        smo = SMO(X_train, y_train_floats, kernel, C)
        alphas, b = smo.run()        
        svm = SVMClassifier(alphas, X_train, y_train_floats, kernel)
        
        # test on acc 
        acc = svm.accuracy(X_val, y_val_floats, b)
        
        if acc > best_acc:
            best_acc = acc
            best_g = g
            best_C = C 
            
        if acc >= 0.9:
            best_accs.append(acc)
            best_gs.append(g)
            best_Cs.append(C)

True labels:  [ 1  1  1  1  1 -1 -1 -1  1  1  1 -1  1 -1 -1  1 -1  1  1 -1]
Testing for C = 0.1, g = 1e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
num correct 15, accuracy 0.5555555555555556
Testing for C = 0.1, g = 1.6237767391887243e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
num correct 15, accuracy 0.5555555555555556
Testing for C = 0.1, g = 2.6366508987303556e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
num correct 15, accuracy 0.5555555555555556
Testing for C = 0.1, g = 4.281332398719396e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [1

shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 26, accuracy 0.9629629629629629
Testing for C = 1.0, g = 2.0691380811147902e-08
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 27, accuracy 1.0
Testing for C = 1.0, g = 3.3598182862837814e-08
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 27, accuracy 1.0
Testing for C = 1.0, g = 5.4555947811685146e-08
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. 

shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 25, accuracy 0.9259259259259259
Testing for C = 100.0, g = 1.6237767391887243e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 26, accuracy 0.9629629629629629
Testing for C = 100.0, g = 2.6366508987303556e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 26, accuracy 0.9629629629629629
Testing for C = 100.0, g = 4.281332398719396e-10
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  

shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 27, accuracy 1.0
Testing for C = 1000.0, g = 1.274274985703132e-08
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 27, accuracy 1.0
Testing for C = 1000.0, g = 2.0691380811147902e-08
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1.
 -1.  1. -1. -1. -1.  1.  1.  1. -1.]
num correct 27, accuracy 1.0
Testing for C = 1000.0, g = 3.3598182862837814e-08
shape of X_test (27, 784) and shape of kernel mat (27, 153), rows should match
pred (27,), [-1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.

In [50]:
print(best_C)
print(best_g)
print(best_acc)

0.1
1.438449888287663e-07
1.0


In [58]:
# retrain on whole dataset with best g and best C. test on test set

print("True labels: ", y_test_floats)
print("Using gamma = {} and C = {}".format(best_g, best_C))

kernel = kernel_rbf(best_g)
smo = SMO(whole_X_train, whole_y_train_floats, kernel, best_C)
alphas, b = smo.run()   

svm = SVMClassifier(alphas, whole_X_train, whole_y_train_floats, kernel)

# test on acc 
acc = svm.accuracy(X_test, y_test_floats, b)
print("Test set acc: {}".format(acc))

True labels:  [ 1  1  1 -1  1  1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  1  1 -1  1  1  1 -1
 -1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1 -1  1  1 -1  1  1
 -1 -1]
Using gamma = 1.438449888287663e-07 and C = 0.1
shape of X_test (50, 784) and shape of kernel mat (50, 450), rows should match
pred (50,), [ 1.  1.  1. -1.  1.  1.  1. -1. -1.  1. -1.  1. -1.  1. -1. -1.  1.  1.
  1. -1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1. -1.
  1.  1.  1. -1. -1. -1. -1.  1.  1. -1.  1.  1. -1. -1.]
num correct 48, accuracy 0.96
Test set acc: 0.96


In [53]:
## yo this shit works! 

### Interior Point, Log Barrier, Gaussian Kernel 

In [309]:
barrier = Barrier()
# C = 1
# gamma = 1e-7
# # gamma = 1 / (X_train.shape[0] * X_train.var())
# print(gamma) 

In [346]:
best_acc = float('-inf')
best_g = None
best_C = None 

best_accs = []
best_gs = []
best_Cs = []

m = 2 * y_train_floats.shape[0] # number of constraints, two per Lagrange multiplier 
print("True labels: ", y_test_floats)
for C in np.logspace(-1, 3, num=5):
    for g in np.logspace(-10, -6, num = 20):
        kernel = kernel_rbf(g)
        lagrange_multipliers = barrier.run(X_train, y_train_floats, kernel, C, m)
        svm = SVMClassifier(lagrange_multipliers, X_train, y_train_floats, kernel)
        
        # test on acc 
        acc = svm.accuracy(X_val, y_val_floats)
        
        if acc > best_acc:
            best_acc = acc
            best_g = g
            best_C = C 
            
        if acc >= 0.9:
            best_accs.append(acc)
            best_gs.append(g)
            best_Cs.append(C)

True labels:  [ 1  1  1 -1  1  1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  1  1 -1  1  1  1 -1
 -1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1 -1  1  1 -1  1  1
 -1 -1]
Feasible starting point: [[0.04764398]
 [0.04764398]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.05235602]
 [0.04764398]
 [0.05235602]
 [0.05235602]
 [0.04764398]
 [0.04764398]
 [0.04764398]
 [0.04764398]
 [0.04764

/Users/aneesh/Documents/Asp/Marshall/UCL/Numerical Optimisation/final/log_barrier/svm.py:82: LinAlgWarning: Ill-conditioned matrix (rcond=8.55828e-39): result may not be accurate.
  solved = scipy.linalg.solve(KKT_mat, rhs)


Finished iter: 20
Finished iter: 21
Finished iter: 22
Finished iter: 23
Finished iter: 24
Finished iter: 25
Finished iter: 26


/Users/aneesh/Documents/Asp/Marshall/UCL/Numerical Optimisation/final/log_barrier/svm.py:82: LinAlgWarning: Ill-conditioned matrix (rcond=5.41916e-45): result may not be accurate.
  solved = scipy.linalg.solve(KKT_mat, rhs)


Finished iter: 27
Final a: [[0.08351376]
 [0.09543634]
 [0.08374346]
 [0.08839787]
 [0.09473853]
 [0.09734884]
 [0.0896319 ]
 [0.08908983]
 [0.0974213 ]
 [0.08824259]
 [0.08814951]
 [0.09650931]
 [0.08872336]
 [0.08596349]
 [0.0958225 ]
 [0.09644147]
 [0.09606166]
 [0.09830577]
 [0.09988377]
 [0.08989727]
 [0.08419051]
 [0.08822699]
 [0.09743386]
 [0.09699231]
 [0.09671091]
 [0.08611533]
 [0.08978077]
 [0.08546374]
 [0.09785465]
 [0.09877399]
 [0.09588   ]
 [0.08665843]
 [0.08839926]
 [0.08849302]
 [0.09622515]
 [0.09887201]
 [0.09739078]
 [0.08825162]
 [0.08814588]
 [0.09619504]
 [0.09091258]
 [0.09146999]
 [0.09795996]
 [0.09643022]
 [0.08673347]
 [0.09822263]
 [0.0986259 ]
 [0.09663387]
 [0.09131831]
 [0.09803993]
 [0.08870201]
 [0.09757782]
 [0.09702242]
 [0.09035403]
 [0.08833147]
 [0.09240087]
 [0.08348535]
 [0.08959328]
 [0.09731035]
 [0.08367323]
 [0.09718204]
 [0.09677892]
 [0.09691173]
 [0.0973212 ]
 [0.0976822 ]
 [0.08957933]
 [0.0895379 ]
 [0.09693436]
 [0.09703274]
 [0.097

...

/Users/aneesh/Documents/Asp/Marshall/UCL/Numerical Optimisation/final/log_barrier/svm.py:82: LinAlgWarning: Ill-conditioned matrix (rcond=1.15966e-49): result may not be accurate.
  solved = scipy.linalg.solve(KKT_mat, rhs)


Finished iter: 4
Finished iter: 5
Finished iter: 6
Finished iter: 7
Finished iter: 8
Finished iter: 9
Finished iter: 10
Finished iter: 11
Finished iter: 12


/Users/aneesh/Documents/Asp/Marshall/UCL/Numerical Optimisation/final/log_barrier/svm.py:82: LinAlgWarning: Ill-conditioned matrix (rcond=3.48259e-51): result may not be accurate.
  solved = scipy.linalg.solve(KKT_mat, rhs)


Finished iter: 13
Finished iter: 14
Finished iter: 15
Finished iter: 16
Finished iter: 17
Finished iter: 18
Finished iter: 19
Finished iter: 20
Finished iter: 21
Finished iter: 22
Finished iter: 23
Finished iter: 24
Finished iter: 25
Finished iter: 26
Finished iter: 27
Final a: [[9.29285847e-01]
 [1.69358273e+00]
 [1.01703071e+00]
 [1.23199187e+00]
 [3.16591931e+00]
 [3.19945078e+00]
 [2.80557161e+00]
 [1.43907393e+00]
 [3.35454362e+00]
 [2.02271011e+00]
 [1.58631497e+00]
 [3.28110417e+00]
 [1.15610350e+00]
 [1.07665552e+00]
 [3.27398543e+00]
 [2.90431407e+00]
 [3.46149296e+00]
 [2.93142082e+00]
 [3.98376107e+00]
 [2.12778727e+00]
 [9.55255341e-01]
 [1.25601008e+00]
 [3.41964194e+00]
 [3.29276725e+00]
 [3.47380272e+00]
 [1.65718000e+00]
 [1.79426069e+00]
 [1.00953079e+00]
 [3.10254309e+00]
 [3.54960335e+00]
 [3.12573734e+00]
 [9.29020273e-01]
 [1.31313409e+00]
 [1.66935119e+00]
 [2.81999557e+00]
 [3.69664769e+00]
 [3.24951792e+00]
 [1.82369674e+00]
 [1.36305004e+00]
 [3.18585576e+00]
 

/Users/aneesh/Documents/Asp/Marshall/UCL/Numerical Optimisation/final/log_barrier/svm.py:82: LinAlgWarning: Ill-conditioned matrix (rcond=1.71821e-75): result may not be accurate.
  solved = scipy.linalg.solve(KKT_mat, rhs)


In [347]:
print(best_C)
print(best_g)
print(best_acc)

0.1
5.4555947811685146e-08
0.9558823529411765


In [348]:
# retrain on whole dataset with best g and best C. test on test set

m = 2 * y_train_floats.shape[0] # number of constraints, two per Lagrange multiplier 
print("True labels: ", y_test_floats)
print("Using gamma = {} and C = {}".format(best_g, best_C))

kernel = kernel_rbf(best_g)
lagrange_multipliers = barrier.run(whole_X_train, whole_y_train_floats, kernel, best_C, m)
svm = SVMClassifier(lagrange_multipliers, whole_X_train, whole_y_train_floats, kernel)

# test on acc 
acc = svm.accuracy(X_test, y_test_floats)
print("Test set acc: {}".format(acc))

True labels:  [ 1  1  1 -1  1  1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  1  1 -1  1  1  1 -1
 -1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1 -1 -1  1  1 -1  1  1
 -1 -1]
Using gamma = 5.4555947811685146e-08 and C = 0.1
Feasible starting point: [[0.05377778]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.05377778]
 [0.04622222]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.05377778]
 [0.04622222]
 [0.05377778]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.04622222]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.04622222]
 [0.04622222]
 [0.04622222]
 [0.04622222]
 [0.04622222]
 [0.05377778]
 [0.04622222]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.04622222]
 [0.05377778]
 [0.04622222]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [0.05377778]
 [

In [350]:
# YO IT WORKS !!! 